# Лабораторная работа 3


**Цель работы:** продолжение изучения основных способов обработки табличных данных и создания запросов к ним с помощью модуля `pandas`.

**Краткое описание:** в лабораторной работе представлено 9 заданий на сортировку, фильтрацию и группировку данных методами модуля `pandas`.

Файл с лабораторной работой находится по ссылке:
https://colab.research.google.com/drive/15mhrnPOtTMvRD7gNG8wHVWE7yciGZGOv?usp=sharing

# Данные


В папке [ЛАБ 3 Анализ данных](https://drive.google.com/drive/folders/1TNdeIqN4Gtq95lkYcMk3FZm8T-bcoTGR?usp=drive_link) расположено несколько таблиц с обезличенными транзакционными банковскими данными. Все задания лабораторной работы 3 необходимо выполнять по этим данным.

### Таблица ```transactions.csv```


##### **Описание**
Таблица содержит историю транзакций клиентов банка за один год и три месяца.

##### **Формат данных**

```
customer_id,tr_datetime,mcc_code,tr_type,amount,term_id
111111,15 01:40:52,1111,1000,-5224,111111
111112,15 15:18:32,3333,2000,-100,11122233
...
```
##### **Описание полей**

 - ```customer_id``` — идентификатор клиента;
 - ```tr_datetime``` — день и время совершения транзакции (дни нумеруются с начала данных);
 - ```mcc_code``` — mcc-код транзакции;
 - ```tr_type``` — тип транзакции;
 - ```amount``` — сумма транзакции в условных единицах со знаком:
  - ```+``` — начисление средств клиенту (приходная транзакция);
  - ```-``` — списание средств (расходная транзакция);
 - ```term_id``` — идентификатор терминала.


 ### Таблица ```gender_train.csv```



##### **Описание**
Таблица содержит информацию по полу для части клиентов, для которых он известен. Для остальных клиентов пол неизвестен.

##### **Формат данных**
```
customer_id,gender
111111,0
111112,1
...
```

##### **Описание полей**
 - ```customer_id``` — идентификатор клиента;
 - ```gender``` — пол клиента.

### Таблица ```tr_mcc_codes.csv```



##### **Описание**
Таблица содержит описание mcc-кодов транзакций.

##### **Формат данных**
```
mcc_code;mcc_description
1000;словесное описание mcc-кода 1000
2000;словесное описание mcc-кода 2000
...
```

##### **Описание полей**
 - ```mcc_code``` – mcc-код транзакции;
 - ```mcc_description``` — описание mcc-кода транзакции.

### Таблица ```tr_types.csv```



##### **Описание**
Таблица содержит описание типов транзакций.

##### **Формат данных**
```
tr_type;tr_description
1000;словесное описание типа транзакции 1000
2000;словесное описание типа транзакции 2000
...
```

##### **Описание полей**
 - ```tr_type``` – тип транзакции;
 - ```tr_description``` — описание типа транзакции.

# Практические задания

In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.float_format', '{:.0f}'.format) # Отключение экспоненциального формата отображения

## Подготовка

Загрузите и прочитайте данные из таблиц в переменные **tr_mcc_codes, tr_types, transactions** и **gender_train** из одноименных таблиц.

Выведите размеры таблиц и случайные строки из них.


In [2]:
tr_types = pd.read_csv("/tr_types.csv", sep=";")          # Разделитель ";"
tr_mcc_codes = pd.read_csv("/tr_mcc_codes.csv", sep=";")  # Разделитель ";"
transactions = pd.read_csv("/transactions.csv",sep=None, engine="python") # Разделитель " ", то ли табуляция, то ли пробел
gender_train = pd.read_csv("/gender_train.csv")           # Разделитель ",", можно оставить как есть

#### Размеры датафреймов:

In [3]:
print(" Размер tr_types:     ", tr_types.shape, "\n",
      "Размер tr_mcc_codes: ", tr_mcc_codes.shape,"\n",
      "Размер tr_mcc_codes: ", transactions.shape,"\n",
      "Размер gender_train: ", gender_train.shape)

 Размер tr_types:      (155, 2) 
 Размер tr_mcc_codes:  (184, 2) 
 Размер tr_mcc_codes:  (389127, 6) 
 Размер gender_train:  (8400, 2)


#### Случайные строки из датафреймов

In [4]:
tr_types.sample(5)

,tr_type,tr_description
59,7070,Перевод на карту (с карты) через Мобильный бан...
69,8000,Поправки по счету клиен. Зачисл.
74,8050,Перевод средств с карты Visa Classic (E/M Stan...
132,2401,Перевод средств с карты на счет клиента через АТМ
113,2210,Наличные. АТМ Зарубеж. банк


In [5]:
tr_mcc_codes.sample(5)

,mcc_code,mcc_description
58,5441,Кондитерские
54,5331,Универсальные магазины
81,5713,Покрытия для пола
125,6012,Финансовые институты — торговля и услуги
149,7538,СТО общего назначения


In [6]:
transactions.sample(5)

,customer_id,tr_datetime,mcc_code,tr_type,amount,term_id
136995,56420506,231 13:31:59,4829,2370,-336887,888990
25507,96661157,101 13:54:60,4829,2370,-1348,NaN
11852,39472448,428 08:36:17,6011,2010,-13475,970288
15006,64294802,184 00:00:00,6011,2110,-89837,NaN
62975,54920864,104 17:11:14,6011,2011,-11230,NaN


In [7]:
gender_train.sample(5)

,customer_id,gender
538,78203753,1
5976,79211748,0
3477,6314083,1
6685,18104975,1
4173,41599422,0


Задания 1–4 нужно выполнить без использования функции `merge` и аналогичных по назначению.


>Допускается для таблицы **transactions** использование не всех записей. При чтении файлов обратите внимание на разделители внутри каждого из файлов – они могут различаться!


## Задание 1

1. Для столбца `tr_type` датафрейма `transactions` выберите произвольные `1000` строк с помощью метода `sample()`.
2. В полученной на предыдущем этапе подвыборке найдите долю транзакций (стобец `tr_description` в датафрейме `tr_types`), в которой содержится подстрока `'POS'` или `'ATM'`.


Выведем 1000 значений столбца tr_type датафрейма transactions, добавим аргумент random_state для воспроизводимости результатов

In [8]:
transactions_sample = transactions.sample(n=1000, random_state=42)
sample_types = transactions_sample['tr_type']

Находим долю транзакций с подстрокой POS или ATM

In [9]:
# создаём словарь соответствий tr_type → tr_description
type_to_desc = dict(zip(tr_types['tr_type'], tr_types['tr_description']))

# добавляем столбец с описанием в выборку
transactions_sample['tr_description'] = transactions_sample['tr_type'].map(type_to_desc)

# проверяем наличие 'POS' или 'ATM'
mask = transactions_sample['tr_description'].str.contains('POS|ATM', case=False, na=False)
# не учитываем регистр, если Nan считаем как False, то есть неподходящим

# доля таких транзакций
count_pos_atm = mask.sum()           # количество подходящих строк
total_count = len(transactions_sample)
share = count_pos_atm / total_count

# 7. Выводим результат
# print(f"Всего транзакций в выборке: {total_count}")
# print(f"Транзакций с POS или ATM: {count_pos_atm}")
print(f"Доля таких транзакций: {share:.2%}") # отображение не в экспоненциальном формате

Доля таких транзакций: 42.40%


## Задание 2


1. Для столбца `tr_type` датафрейма `transactions` посчитайте частоту встречаемости всех типов транзакций `tr_type` в `transactions`.
2. Выведите датафрейм, содержащий топ-10 транзакций, отсортированных по убыванию частоты встречаемости. Датафрейм должен содержать столбцы `tr_type`, `tr_description` и столбец с частотой встречаемости.

In [10]:
# Считаем частоту встречаемости tr_type
freq = transactions['tr_type'].value_counts()  # вернёт Series: индекс = tr_type, значения = count

In [11]:
# Превращаем в датафрейм и добавляем столбец с частотой
freq_df = freq.reset_index()
freq_df.columns = ['tr_type', 'frequency']

In [12]:
# Добавляем описание
type_to_desc = dict(zip(tr_types['tr_type'], tr_types['tr_description']))
freq_df['tr_description'] = freq_df['tr_type'].map(type_to_desc)

# Сортируем по убыванию частоты и берём топ-10
top10_df = freq_df.sort_values(by='frequency', ascending=False).head(10)
top10_df # Результат

,tr_type,frequency,tr_description
0,1010,95765,Покупка. POS ТУ СБ РФ
1,2010,65545,Выдача наличных в АТМ Сбербанк России
2,1110,62599,Покупка. POS ТУ Россия
3,1030,50959,Оплата услуги. Банкоматы СБ РФ
4,7070,30235,Перевод на карту (с карты) через Мобильный бан...
5,2370,22521,Списание с карты на карту по операции <перевод...
6,7010,11990,Взнос наличных через АТМ (в своем тер.банке)
7,1100,7419,Покупка. ТУ Россия
8,7030,5737,Перевод на карту (с карты) через АТМ (в предел...
9,1200,5589,Покупка. Зарубеж. ТУ


## Задание 3
1. В датафрейме `transactions` найдите клиента с максимальной суммой приходов на карту.
2. В датафрейме `transactions` найдите клиента с максимальной суммой расходов по карте.
3. Найдите модуль разницы для этих клиентов между суммой расходов и суммой приходов.

In [13]:
# Создаём колонки для прихода и расхода
transactions['income'] = np.where(transactions['amount'] > 0, transactions['amount'], 0)
transactions['expense'] = np.where(transactions['amount'] < 0, -transactions['amount'], 0)

# Группируем по клиенту
summary = transactions.groupby('customer_id').agg({'income':'sum', 'expense':'sum'}).reset_index()

# Клиент с максимальной суммой приходов
max_income_client = summary.loc[summary['income'].idxmax()]

# Клиент с максимальной суммой расходов
max_expense_client = summary.loc[summary['expense'].idxmax()]

# Модуль разницы между расходами и приходами
diff_income = abs(max_income_client['income'] - max_income_client['expense'])
diff_expense = abs(max_expense_client['income'] - max_expense_client['expense'])

# Вывод результатов
print("Клиент с максимальным приходом:")
print(max_income_client)
print(f"Модуль разницы (расход - приход): {diff_income}\n")

print("Клиент с максимальным расходом:")
print(max_expense_client)
print(f"Модуль разницы (расход - приход): {diff_expense}")

Клиент с максимальным приходом:
customer_id     19950741
income         512422498
expense       1075056264
Name: 138, dtype: float64
Модуль разницы (расход - приход): 562633765.6400001

Клиент с максимальным расходом:
customer_id     19950741
income         512422498
expense       1075056264
Name: 138, dtype: float64
Модуль разницы (расход - приход): 562633765.6400001


## Задание 4
1. Найдите среднее арифметическое и медиану по столбцу `amount` для каждого типа транзакций из топ-10 в Задании 2.
1. Найдите среднее арифметическое и медиану по столбцу `amount` для каждого типа транзакций, совершенных клиентами из Задания 3.

In [14]:
top10_types = top10_df['tr_type'].tolist() # список из датафрейма по столбцу tr_type

# Фильтруем только транзакции этих типов
top10_transactions = transactions[transactions['tr_type'].isin(top10_types)]

# Группируем по типу транзакции и считаем среднее и медиану
stats_top10 = top10_transactions.groupby('tr_type')['amount'].agg(['mean','median']).reset_index()

# Добавляем описание транзакции из tr_types
type_to_desc = dict(zip(tr_types['tr_type'], tr_types['tr_description']))
stats_top10['tr_description'] = stats_top10['tr_type'].map(type_to_desc)

print("Среднее и медиана для топ-10 типов транзакций:")
stats_top10

Среднее и медиана для топ-10 типов транзакций:


,tr_type,mean,median,tr_description
0,1010,-22927,-8112,Покупка. POS ТУ СБ РФ
1,1030,-5980,-2246,Оплата услуги. Банкоматы СБ РФ
2,1100,-51820,-12105,Покупка. ТУ Россия
3,1110,-34357,-11858,Покупка. POS ТУ Россия
4,1200,-40958,-7389,Покупка. Зарубеж. ТУ
5,2010,-140064,-44918,Выдача наличных в АТМ Сбербанк России
6,2370,-254778,-67377,Списание с карты на карту по операции <перевод...
7,7010,335936,112296,Взнос наличных через АТМ (в своем тер.банке)
8,7030,162421,22459,Перевод на карту (с карты) через АТМ (в предел...
9,7070,149698,16507,Перевод на карту (с карты) через Мобильный бан...


In [15]:
# customer_id клиентов с максимальным приходом и расходом
customers_of_interest = [max_income_client['customer_id'], max_expense_client['customer_id']]

# Фильтруем транзакции этих клиентов
transactions_customers = transactions[transactions['customer_id'].isin(customers_of_interest)]

# Группируем по типу транзакции и считаем среднее и медиану
stats_customers = transactions_customers.groupby('tr_type')['amount'].agg(['mean','median']).reset_index()

# Добавляем описание
stats_customers['tr_description'] = stats_customers['tr_type'].map(type_to_desc)

print("\nСреднее и медиана для транзакций клиентов из задания 3:")
stats_customers


Среднее и медиана для транзакций клиентов из задания 3:


,tr_type,mean,median,tr_description
0,1010,-65668,-20438,Покупка. POS ТУ СБ РФ
1,1030,-26473,-22459,Оплата услуги. Банкоматы СБ РФ
2,1100,-54771,-1774,Покупка. ТУ Россия
3,1110,-45503,-28972,Покупка. POS ТУ Россия
4,1200,-185784,-143090,Покупка. Зарубеж. ТУ
5,1210,-187868,-51396,Покупка. POS Зарубеж. ТУ
6,1410,-34240,-29911,н/д
7,1510,-15183,-15183,н/д
8,2010,-304062,-224592,Выдача наличных в АТМ Сбербанк России
9,2011,-473247,-89837,Выдача наличных в АТМ Сбербанка (в других ТБ)


## Подготовка для заданий 5–8


Соедините датафрейм `transactions` со всеми остальными таблицами (`tr_mcc_codes`, `tr_types`, `gender_train`). Причем объединение с таблицей `gender_train` необходимо выполнить с помощью `left join`, а с оставшимися датафреймами – через `inner`.
После получения общей таблицы `gender_train`, `tr_types` и `tr_mcc_codes` можно удалить. В результате соединения датафреймов должно получиться `999584` строки.

In [16]:
transactions = pd.merge(transactions, gender_train, how='left')
transactions = pd.merge(transactions, tr_mcc_codes, how='inner')
transactions = pd.merge(transactions, tr_types, how='inner')
transactions.shape

(388977, 11)

## Задание 5

1. Определите модуль разницы между средними тратами женщин и мужчин (трата – отрицательное значение в столбце `amount`).
2. Определите модуль разницы между средними поступлениями у мужчин и женщин.

>Обратите внимание, что для вычисления модуля разности не требуется точных знаний о том,
каким значением в таблице обозначены мужчины, а каким – женщины.

Разница между средними тратами

In [17]:
expenses = transactions[transactions['amount'] < 0]

mean_expenses_by_gender = (
    expenses.groupby('gender')['amount']
    .mean()
    .rename(index={0: 'male', 1: 'female'})
    .reset_index()  # превращаем в DataFrame
)

mean_expenses_by_gender.columns = ['gender', 'mean_expense']  # даём имя столбцам

# Разница (теперь доступ по loc)
diff_expenses = abs(
    mean_expenses_by_gender.loc[mean_expenses_by_gender['gender'] == 'male', 'mean_expense'].values[0]
    - mean_expenses_by_gender.loc[mean_expenses_by_gender['gender'] == 'female', 'mean_expense'].values[0]
)

print("Средние траты по полу:")
print(mean_expenses_by_gender)
print(f"\nМодуль разницы между средними тратами: {diff_expenses:}")


Средние траты по полу:
   gender  mean_expense
0    male        -64372
1  female        -90745

Модуль разницы между средними тратами: 26372.516461332147


Разница между средними поступлениями

In [18]:
# Фильтруем только поступления (amount > 0)
incomes = transactions[transactions['amount'] > 0]

# Средние поступления по полу
mean_incomes_by_gender = (
    incomes.groupby('gender')['amount']
    .mean()
    .rename(index={0: 'male', 1: 'female'})
    .reset_index()
)

mean_incomes_by_gender.columns = ['gender', 'mean_expense']  # даём имя столбцам

# Разница (теперь доступ по loc)
diff_incomes = abs(
    mean_incomes_by_gender.loc[mean_incomes_by_gender['gender'] == 'male', 'mean_expense'].values[0]
    - mean_incomes_by_gender.loc[mean_incomes_by_gender['gender'] == 'female', 'mean_expense'].values[0]
)


print("\nСредние поступления по полу:")
print(mean_incomes_by_gender)
print(f"\nМодуль разницы между средними поступлениями: {diff_incomes:.2f}")


Средние поступления по полу:
   gender  mean_expense
0    male        301665
1  female        155416

Модуль разницы между средними поступлениями: 146249.52


## Задание 6

1. Для каждого типа транзакций рассчитайте максимальную сумму прихода на карту (из строго положительных сумм по столбцу `amount`) отдельно для мужчин и женщин (назовите ее `max_income`). Оставьте по 10 типов транзакций для мужчин и для женщин, наименьших среди всех типов транзакций по полученным значениям `max_income`.
2. Среди типов транзакций, найденных в пункте 1, выделите те, которые встречаются одновременно и у мужчин, и у женщин.

In [19]:
# Оставляем только приходные транзакции (amount > 0)
incomes = transactions[transactions['amount'] > 0]

# Считаем максимальную сумму прихода по каждому типу и полу
max_income = (
    incomes
    .groupby(['gender', 'tr_type'])['amount']
    .max()
    .reset_index()
    .rename(columns={'amount': 'max_income'})
)

# Разделяем мужчин и женщин
men_max_income = max_income[max_income['gender'] == 1]
women_max_income = max_income[max_income['gender'] == 0]

# Для каждого пола выбираем 10 типов с наименьшим max_income
men_bottom10 = men_max_income.nsmallest(10, 'max_income')
women_bottom10 = women_max_income.nsmallest(10, 'max_income')

# Находим типы, которые встречаются у обоих полов
common_types = set(men_bottom10['tr_type']).intersection(women_bottom10['tr_type'])

# Формируем итоговые датафреймы
men_bottom10_common = men_bottom10[men_bottom10['tr_type'].isin(common_types)]
women_bottom10_common = women_bottom10[women_bottom10['tr_type'].isin(common_types)]

print("🔹 10 типов с наименьшими max_income (мужчины):")
print(men_bottom10)

print("\n🔹 10 типов с наименьшими max_income (женщины):")
print(women_bottom10)

print("\n🔹 Общие типы транзакций для обоих полов:")
print(common_types)

🔹 10 типов с наименьшими max_income (мужчины):
    gender  tr_type  max_income
40       1     4110        2246
33       1     1010       28804
37       1     1210       31443
38       1     2010       44918
41       1     6000       79281
59       1     7041       87591
63       1     7075       89837
39       1     2110      112296
50       1     7015      121279
34       1     1100      163727

🔹 10 типов с наименьшими max_income (женщины):
    gender  tr_type  max_income
7        0     4110        2246
8        0     4210        2246
6        0     4100        5042
31       0     7075        6738
4        0     2110       22459
14       0     6210       31751
3        0     2010       33689
26       0     7040       47274
25       0     7035       49410
5        0     2210       68316

🔹 Общие типы транзакций для обоих полов:
{2110.0, 2010.0, 7075.0, 4110.0}


## Задание 7

1. Найдите суммы затрат по каждой категории (`mcc`) для мужчин и для женщин.
2. Найдите топ-10 категорий с самыми большими относительными модулями разности в тратах для разных полов (в ответе должны присутствовать описания mcc-кодов).

In [29]:
# Берем только расходы (amount < 0)
expenses = transactions[transactions['amount'] < 0].copy()

# Считаем сумму расходов по каждой категории и полу
expenses_sum = (
    expenses
    .groupby(['gender', 'mcc_code'])['amount']
    .sum()
    .reset_index()
)

# Разворачиваем таблицу: отдельно колонки для мужчин и женщин
expenses_pivot = expenses_sum.pivot(index='mcc_code', columns='gender', values='amount').fillna(0)

# Переименуем для наглядности
expenses_pivot.columns = ['female_sum', 'male_sum']  # предполагаем, что 0 = женщина, 1 = мужчина

# Находим модуль относительной разницы в тратах
# (разница / среднее по модулю)
expenses_pivot['abs_diff_ratio'] = (
    (expenses_pivot['male_sum'] - expenses_pivot['female_sum']).abs() /
    ((expenses_pivot['male_sum'].abs() + expenses_pivot['female_sum'].abs()) / 2)
)

# Оставляем только топ-10 категорий с максимальной относительной разницей
top10_diff = (
    expenses_pivot
    .sort_values('abs_diff_ratio', ascending=False)
    .head(10)

    .reset_index()
)

# Добавляем описания mcc-кодов
top10_diff = top10_diff.merge(tr_mcc_codes, on='mcc_code', how='left')

# Выводим результат
print("🔹 Топ-10 категорий с наибольшей относительной разницей в тратах между мужчинами и женщинами:")
top10_diff[['mcc_code', 'mcc_description', 'male_sum', 'female_sum', 'abs_diff_ratio']]

🔹 Топ-10 категорий с наибольшей относительной разницей в тратах между мужчинами и женщинами:


,mcc_code,mcc_description,male_sum,female_sum,abs_diff_ratio
0,742,Ветеринарные услуги,0,-203570,2
1,1711,"Генеральные подрядчики по вентиляции, теплосна...",-105041,0,2
2,5065,Электрические части и оборудование,-23223,0,2
3,5172,Нефть и нефтепродукты,-57720,0,2
4,5192,"Книги, периодические издания и газеты",-42717,0,2
5,5131,"Штучные товары, галантерея и другие текстильны...",0,-79337,2
6,5013,Поставщики грузовиков и запчастей,-1271381,0,2
7,5072,Оборудование и сопутствующие материалы для тех...,-610088,0,2
8,7629,"Ремонт бытовой техники, ремонт электроприборы",0,-138752,2
9,5733,"Продажа музыкальных инструментов, фортепиано, нот",-772706,0,2


## Задание 8

1. Из поля `tr_datetime` выделите час `tr_hour`, в который произошла транзакция, как первые 2 цифры до `":"`.
2. Посчитайте количество транзакций со значением в столбце `amount` строго меньше `0` в ночное время для мужчин и женщин. Ночное время – это промежуток с 00:00 по 06:00 часов.

In [47]:
# Извлекаем час из поля tr_datetime (первые два символа до ":")
transactions['tr_hour'] = transactions['tr_datetime'].str.extract(r'(\d{2}):')[0].astype(int)

# Фильтруем только расходы (amount < 0)
expenses = transactions[transactions['amount'] < 0]

# Отбираем только ночные транзакции (с 00 по 06 час включительно)
night_expenses = expenses[expenses['tr_hour'].between(0, 6, inclusive='both')].copy()

night_expenses['gender'] = night_expenses['gender'].replace({0: 'male', 1: 'female'})
# Считаем количество таких транзакций по полу
night_count_by_gender = (
    night_expenses
    .groupby('gender')
    .size()
    .reset_index(name='count')
)

# Выводим результат
print("Количество ночных расходных транзакций по полу:")

night_count_by_gender

Количество ночных расходных транзакций по полу:


,gender,count
0,female,22817
1,male,17204


## Задание 9

1. Найдите категорию транзакций, в которой совершается больше всего трат в ночное время (промежуток с 00:00 по 06:00). Выведите название категории и среднюю сумму трат в ней.
2. Найдите 3 категории транзакций, в которых средние траты мужчин и женщин различаются меньше всего.
3. Найдите терминал, через который проходит наибольшее число транзакций. Выведите идентификаторы 5 пользователей с наибольшей суммой отрицательных транзакций в этом терминале.

Категория, где больше всего трат ночью

In [48]:
# Выделяем час
transactions['tr_hour'] = transactions['tr_datetime'].str.extract(r'(\d{2}):')[0].astype(int)

# Фильтруем ночные расходы
night_expenses = transactions[(transactions['amount'] < 0) & (transactions['tr_hour'].between(0, 6))]

# Суммируем по категориям (mcc_description)
night_sum_by_cat = (
    night_expenses
    .groupby(['mcc_code', 'mcc_description'], as_index=False)['amount']
    .agg(['count', 'mean', 'sum'])
    .reset_index()
)

# Категория с максимальной суммой трат
top_night_cat = night_sum_by_cat.sort_values('sum').iloc[0]  # min, потому что суммы отрицательные

print("Категория с наибольшими ночными тратами:")
print(f"Название категории: {top_night_cat['mcc_description']}")
print(f"Средняя сумма трат: {abs(top_night_cat['mean']):.2f}")

Категория с наибольшими ночными тратами:
Название категории: Финансовые институты — снятие наличности автоматически
Средняя сумма трат: 129973.98


топ-3 категории, где средние траты мужчин и женщин ближе всего

In [66]:
# Оставляем только корректные значения пола (0 и 1)
transactions = transactions[transactions['gender'].isin([0, 1])]

# И приводим тип к int, если вдруг float
transactions['gender'] = transactions['gender'].astype(int)

avg_expenses_by_gender = (
    transactions[transactions['amount'] < 0]
    .groupby(['mcc_code', 'mcc_description', 'gender'], as_index=False)['amount']
    .mean()
)

pivot_avg = avg_expenses_by_gender.pivot(
    index=['mcc_code', 'mcc_description'],
    columns='gender',
    values='amount'
).rename(columns={0: 'female', 1: 'male'})

pivot_avg['diff'] = (pivot_avg['male'] - pivot_avg['female']).abs()
pivot_avg.columns.name = None

top3_min_diff = pivot_avg.sort_values('diff').head(3).reset_index()

print("Топ-3 категории, где средние траты мужчин и женщин ближе всего:")
top3_min_diff[['mcc_description', 'female', 'male', 'diff']]

Топ-3 категории, где средние траты мужчин и женщин ближе всего:


,mcc_description,female,male,diff
0,Флористика,-25664,-25752,88
1,Кондитерские,-11154,-11680,526
2,Видеопрокат,-2605,-3222,616


Терминал, с наибольшим числом транзакций. Топ-5 клиентов по расходам

In [67]:
# Находим терминал с наибольшим числом транзакций
top_terminal = (
    transactions['term_id']
    .value_counts()
    .idxmax()
)

print(f"Терминал с наибольшим количеством транзакций: {top_terminal}")

# Фильтруем все транзакции по этому терминалу
top_term_tx = transactions[transactions['term_id'] == top_terminal]

# Суммируем отрицательные транзакции (траты) по клиентам
top_customers = (
    top_term_tx[top_term_tx['amount'] < 0]
    .groupby('customer_id', as_index=False)['amount']
    .sum()
    .sort_values('amount')
    .head(5)
)

print("Топ-5 клиентов с наибольшими расходами в этом терминале:")
top_customers

Терминал с наибольшим количеством транзакций: 888993
Топ-5 клиентов с наибольшими расходами в этом терминале:


,customer_id,amount
37,19950741,-27355591
111,60293458,-11170890
15,9497605,-10543002
43,21276897,-9868778
97,52847004,-8691694


# Порядок защиты работы

**Для защиты работы необходимо:**
1. Предоставить результаты выполнения 9 заданий в виде ссылки на Google Colab.

# Примечание

Для тех кто чувствует в себе силы рекомендуется выполнение некоторых заданий двумя способами – с помощью `pandas` и с помощью `SQL` с использованием модуля `sqlite3`.

# Дополнительные материалы

1. Pandas. User Guide [Электронный ресурс]. URL: https://pandas.pydata.org/pandas-docs/stable/user_guide/index.html (дата обращения: 14.03.2024).
2. МакКинни, У. Python и анализ данных [Текст] / У. МакКинни ; пер. с англ. Слинкиной А. А. — Москва : ДМК-Пресс, 2023. — 536 с.
3. Хейдт, М. Изучаем pandas [Текст] / М. Хейдт. — Москва : ДМК Пресс, 2019. — 700 с.